# Deploy the finetuned vicuna model on Amazon SageMaker

As we have finetuned the model, next we will show you how to deploy the model on SageMaker.

In this notebook, we explore how to host a large language model on SageMaker using the [Large Model Inference](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-large-model-inference.html) container that is optimized for hosting large models using DJLServing. DJLServing is a high-performance universal model serving solution powered by the Deep Java Library (DJL) that is programming language agnostic. To learn more about DJL and DJLServing, you can refer to our recent [blog post](https://aws.amazon.com/blogs/machine-learning/deploy-large-models-on-amazon-sagemaker-using-djlserving-and-deepspeed-model-parallel-inference/).

## Create a SageMaker Model for Deployment
As a first step, we'll import the relevant libraries and configure several global variables such as the hosting image that will be used nd the S3 location of our model artifacts

In [ ]:
import sagemaker
from sagemaker.model import Model
from sagemaker import serializers, deserializers
from sagemaker import image_uris
import boto3
import os
import time
import json
import jinja2
from pathlib import Path

In [ ]:
role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

region = sess._region_name # region name of the current SageMaker Studio environment
account_id = sess.account_id()  # account_id of the current SageMaker Studio environment

s3_client = boto3.client("s3") # client to intreract with S3 API
sm_client = boto3.client("sagemaker")  # client to intreract with SageMaker
smr_client = boto3.client("sagemaker-runtime") # client to intreract with SageMaker Endpoints
jinja_env = jinja2.Environment() # jinja environment to generate model configuration templates

In [23]:
# lookup the inference image uri based on our current region
djl_inference_image_uri = (
    f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.3-cu117"
    #f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.22.1-deepspeed0.9.2-cu118"
)

## Build the inference contianer image

In [24]:
%%writefile Dockerfile.inference
## You should change below region code to the region you used, here sample is use us-west-2
From 763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.3-cu117 

ENV LANG=C.UTF-8
ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE

## Install transfomers version which support LLaMaTokenizer
#RUN python3 -m pip install git+https://github.com/huggingface/transformers.git@68d640f7c368bcaaaecfc678f11908ebbd3d6176
## Install transfomers version which support vicuna v1.1 LLaMaTokenizer
RUN python3 -m pip install transformers==4.28.1
#RUN python3 -m pip install transformers==4.28.0.dev0

## Make all local GPUs visible
ENV NVIDIA_VISIBLE_DEVICES="all"

Overwriting Dockerfile.inference


In [25]:
## You should change below region code to the region you used, here sample is use us-west-2
!aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-west-2.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [26]:
## define repo name, should contain *sagemaker* in the name
repo_name = "sagemaker-vicuna-v11-inference-demo"

In [27]:
%%script env repo_name=$repo_name bash

#!/usr/bin/env bash

# This script shows how to build the Docker image and push it to ECR to be ready for use
# by SageMaker.

# The argument to this script is the image name. This will be used as the image on the local
# machine and combined with the account and region to form the repository name for ECR.
# The name of our algorithm
algorithm_name=${repo_name}

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} -f Dockerfile.inference .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded
Sending build context to Docker daemon  68.13GB
Step 1/6 : From 763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.3-cu117
 ---> b56f005c56a2
Step 2/6 : ENV LANG=C.UTF-8
 ---> Using cache
 ---> 0da52e636fda
Step 3/6 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> 8c1d9f232cf2
Step 4/6 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache
 ---> 8fda7737cf66
Step 5/6 : RUN python3 -m pip install transformers==4.28.1
 ---> Using cache
 ---> 5449aa64fce1
Step 6/6 : ENV NVIDIA_VISIBLE_DEVICES="all"
 ---> Using cache
 ---> 93bb8ecf9dce
Successfully built 93bb8ecf9dce
Successfully tagged sagemaker-vicuna-v11-inference-demo:latest
The push refers to repository [687912291502.dkr.ecr.us-west-2.amazonaws.com/sagemaker-vicuna-v11-inference-demo]
80876c826db4: Preparing
9b7c02d52507: Preparing
2b3933279de9: Preparing
706007d86e67: Preparing
f0f8b6b8d542: Preparing
e74db00f08b1: Preparing
25ec89403616: Preparing
cdc59fe04b2e: Preparing
6a1670674acc: Pre

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [28]:
## The image uri which is build and pushed above
inference_image_uri = "{}.dkr.ecr.{}.amazonaws.com/{}:latest".format(account_id, region, repo_name)
inference_image_uri

'687912291502.dkr.ecr.us-west-2.amazonaws.com/sagemaker-vicuna-v11-inference-demo:latest'

## Deploying a Large Language Model using Hugging Face Accelerate
The DJL Inference Image which we will be utilizing ships with a number of built-in inference handlers for a wide variety of tasks including:
- `text-generation`
- `question-answering`
- `text-classification`
- `token-classification`

You can refer to this [GitRepo](https://github.com/deepjavalibrary/djl-serving/tree/master/engines/python/setup/djl_python) for a list of additional handlers and available NLP Tasks. <br>
These handlers can be utilized as is without having to write any custom inference code. We simply need to create a `serving.properties` text file with our desired hosting options and package it up into a `tar.gz` artifact.

Lets take a look at the `serving.properties` file that we'll be using for our first example

In [29]:
!mkdir accelerate_src

mkdir: cannot create directory ‘accelerate_src’: File exists


In [37]:
%%writefile accelerate_src/serving.properties
engine=Python
option.entryPoint=djl_python.huggingface
#option.s3url={{ s3url }}
#option.model_id=pinkmanlove/llama-7b-hf
#option.model_id=eachadea/vicuna-7b-1.1
option.model_id=TheBloke/Llama-2-7B-fp16
option.task=text-generation
option.device_map=auto
option.load_in_8bit=TRUE

Overwriting accelerate_src/serving.properties


There are a few options specified here. Lets go through them in turn<br>
1. `engine` - specifies the engine that will be used for this workload. In this case we'll be hosting a model using the [DJL Python Engine](https://github.com/deepjavalibrary/djl-serving/tree/master/engines/python)
2. `option.entryPoint` - specifies the entrypoint code that will be used to host the model. djl_python.huggingface refers to the `huggingface.py` module from [djl_python repo](https://github.com/deepjavalibrary/djl-serving/tree/master/engines/python/setup/djl_python).  
3. `option.s3url` - specifies the location of the model files. Alternativelly an `option.model_id` option can be used instead to specifiy a model from Hugging Face Hub (e.g. `EleutherAI/gpt-j-6B`) and the model will be automatically downloaded from the Hub. The s3url approach is recommended as it allows you to host the model artifact within your own environment and enables faster deployments by utilizing optimized approach within the DJL inference container to transfer the model from S3 into the hosting instance 
4. `option.task` - This is specific to the `huggingface.py` inference handler and specifies for which task this model will be used
5. `option.device_map` - Enables layer-wise model partitioning through [Hugging Face Accelerate](https://huggingface.co/docs/accelerate/usage_guides/big_modeling#designing-a-device-map). With `option.device_map=auto`, Accelerate will determine where to put each **layer** to maximize the use of your fastest devices (GPUs) and offload the rest on the CPU, or even the hard drive if you don’t have enough GPU RAM (or CPU RAM). Even if the model is split across several devices, it will run as you would normally expect.
6. `option.load_in_8bit` - Quantizes the model weights to int8 thereby greatly reducing the memory footprint of the model from the initial FP32. See this [blog post](https://huggingface.co/blog/hf-bitsandbytes-integration) from Hugging Face for additional information 

For more information on the available options, please refer to the [SageMaker Large Model Inference Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-large-model-configuration.html)

Our initial approach here is to utilize the built-in functionality within Hugging Face Transformers to enable Large Language Model hosting. 

In [38]:
%%writefile accelerate_src/requirements.txt
protobuf==3.20
accelerate=>0.17.0

Overwriting accelerate_src/requirements.txt


We place the `serving.properties` file into a tarball and upload it to S3

In [39]:
!tar czvf acc_model.tar.gz accelerate_src/ 

accelerate_src/
accelerate_src/.ipynb_checkpoints/
accelerate_src/.ipynb_checkpoints/serving-checkpoint.template
accelerate_src/.ipynb_checkpoints/requirements-checkpoint.txt
accelerate_src/.ipynb_checkpoints/serving-checkpoint.properties
accelerate_src/serving.template
accelerate_src/serving.properties
accelerate_src/requirements.txt


In [40]:
s3_code_prefix = "llama/deploy/code"

code_artifact = sess.upload_data("acc_model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {code_artifact}")

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-us-west-2-687912291502/llama/deploy/code/acc_model.tar.gz


## Deploy Model to a SageMaker Endpoint
With a helper function we can now deploy our endpoint and invoke it with some sample inputs

In [41]:
def deploy_model(image_uri, model_data, role, endpoint_name, instance_type, sagemaker_session):
    
    """Helper function to create the SageMaker Endpoint resources and return a predictor"""
    model = Model(
            image_uri=image_uri, 
              model_data=model_data, 
              role=role
             )
    
    model.deploy(
        initial_instance_count=1,
        instance_type=instance_type,
        endpoint_name=endpoint_name
        )
    
    # our requests and responses will be in json format so we specify the serializer and the deserializer
    predictor = sagemaker.Predictor(
        endpoint_name=endpoint_name, 
        sagemaker_session=sagemaker_session, 
        serializer=serializers.JSONSerializer(), 
        deserializer=deserializers.JSONDeserializer())
    
    return predictor

In [42]:
# creates a unique endpoint name
endpoint_name = sagemaker.utils.name_from_base("vicuna-7B")
print(f"Our endpoint will be called {endpoint_name}")

Our endpoint will be called vicuna-7B-2023-07-21-07-21-41-136


In [43]:
# deployment will take about 10 minutes
predictor = deploy_model(image_uri=inference_image_uri, 
                            model_data=code_artifact, 
                            role=role, 
                            endpoint_name=endpoint_name, 
                            instance_type="ml.g5.4xlarge", 
                            sagemaker_session=sess)

----------!

Let's run an example with a basic text generation prompt Large model inference is

In [44]:
import boto3
import json
import uuid
client = boto3.client('sagemaker-runtime')
endpoint_name = 'vicuna-7B-2023-07-21-07-21-41-136'
payload = '{"inputs" : "a happy weekend with my family", \
            "parameters": { "max_length": 200, "temperature": 0.6 }  \
           }'
encoded_inp = (payload).encode("utf-8")
response = client.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType='application/json',
        Accept='application/json',
        Body=payload
)


#predictor.predict({ 
#                    "inputs" : "large model inference is", 
#                    "parameters": { "max_length": 100, "temperature": 0.8 }
#                })

/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [45]:
result = response['Body'].read()
result = json.loads(result)
print(result)


[{'generated_text': "a happy weekend with my family.\nI'm so glad you had a good weekend. I hope you have a great week.\nI'm glad you had a good weekend. I hope you have a great week.\nI'm glad you had a good weekend. I hope you have a great week.\nI'm glad you had a good weekend. I hope you have a great week.\nI'm glad you had a good weekend. I hope you have a great week.\nI'm glad you had a good weekend. I hope you have a great week.\nI'm glad you had a good weekend. I hope you have a great week.\nI'm glad you had a good weekend. I hope you have a great week.\nI'm glad you had a good weekend. I hope you have a great week.\nI'm glad you had a good weekend"}]


In [46]:
print(predictor.predict({ "inputs":
                                """Message: Support has been terrible for 2 weeks...
                                Sentiment: Negative
                                ###
                                Message: I love your API, it is simple and so fast!
                                Sentiment: Positive
                                ###
                                Message: GPT-J has been released 12 months ago.
                                Sentiment: Neutral
                                ###
                                Message: The responsiveness of your team has been amazing, thank you so much!
                                Sentiment:""",
                      "parameters": { "max_length": 50, "temperature": 0.5 }
                     }
                    )[0]["generated_text"])

Message: Support has been terrible for 2 weeks...
                                Sentiment: Negative
                                ###
                                Message: I love your API, it is simple and so fast!
                                Sentiment: Positive
                                ###
                                Message: GPT-J has been released 12 months ago.
                                Sentiment: Neutral
                                ###
                                Message: The responsiveness of your team has been amazing, thank you so much!
                                Sentiment: Pos


Finally Let's do a quick benchmark to see what kind of latency we can expect from this model

In [47]:
predictor.predict({ 
                    "inputs" : "a happy weekend with my family", 
                    "parameters": { "max_length": 150, "temperature": 0.5 }
                })

[{'generated_text': "a happy weekend with my family.\nI'm so glad you had a good weekend. I hope you have a great week.\nI'm glad you had a good weekend. I hope you have a great week.\nI'm glad you had a good weekend. I hope you have a great week.\nI'm glad you had a good weekend. I hope you have a great week.\nI'm glad you had a good weekend. I hope you have a great week.\nI'm glad you had a good weekend. I hope you have a great week.\nI'm glad you had a good weekend. I hope you have a great week.\n"}]

In [ ]:
# Clean up the endpoint before proceeding
predictor.delete_endpoint()

## Reference

[sagemaker-hosting/Large-Language-Model-Hosting/](https://github.com/aws-samples/sagemaker-hosting/tree/main/Large-Language-Model-Hosting)